In [31]:
import cv2
import numpy as np
import copy
import math
import os
import keras
import tensorflow as tf
import imutils
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [32]:
def loadModel():
    
    version = 0
    modelDir = "Model Versions/HandModelV"

    # This method always gets the most up to date model.
    
    while True:
        try:
            version = version + 1
            f = open(modelDir + str(version) + ".h5", 'r')
            f.close()
        except:
            break
            
    model = keras.models.load_model(modelDir + str(version-1) + ".h5")
    
    print("Using model " + modelDir + str(version-1) + ".h5")
    
    return model

model = loadModel()

Using model Model Versions/HandModelV5.h5


In [33]:
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

version = 1
dir = "../../../../HandsData/OpenCVHandsData/None/none"

def saveImage(handBox):

    global version
    
    while True:
        try:
            f = open(dir + str(version) + ".jpg", 'r')
            f.close()
            version = version + 1
        except:
            break

    try:
        cv2.imwrite(dir + str(version) + ".jpg", handBox)
    except:
        pass
    

In [34]:
sizeX = 60
sizeY = 100

def resize(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    h, w, ch = image.shape
    
    zeros = np.zeros((sizeY, sizeX, 3))
    
    tempImage = image.copy()
    
    image = imutils.resize(tempImage, height=sizeY)
    if image.shape[1] > sizeX:
        image = imutils.resize(tempImage, width=sizeX)
    
    zeros[:image.shape[0], :image.shape[1]] = image
    
    
    
    return zeros

In [35]:
outputText = ""
combinedOutput = []

def classifyHand(handBox):
    
    global outputText, combinedOutput
    
    try :
        handBox = resize(handBox)
    except:
        print("Mitigating error") # (Nobody will notice :) )
        return outputText, combinedOutput
    
    handBox = np.array(handBox)
        
    pred_hot = model.predict(np.expand_dims(handBox, axis=0))[0]

    
    if pred_hot[0] > pred_hot[1]:
        outputText = "Index "
    else:
        outputText = "Fist "
    
    
    combinedOutput = ["Fist: " + str(pred_hot[1]), "Index " + str(pred_hot[0])]
    
    return outputText, combinedOutput

In [36]:
def removeBackground(frame):

    background = cv2.createBackgroundSubtractorMOG2(0,50)

    kernel = np.ones((3,3), np.uint8)

    bgMask = background.apply(frame)
    bgMask = cv2.erode(bgMask, kernel, iterations=1)

    return cv2.bitwise_and(frame, frame, mask = bgMask)
    

In [37]:
camera = cv2.VideoCapture(0)

while (True):

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = camera.read()

    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # Smoothing
    
    mask = removeBackground(frame)

    
    # Detecting skin

    hsv = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    skinMask = cv2.inRange(hsv, lower, upper)
    
    
    #Find contours in the image
    
    contours, heirarchy = cv2.findContours(skinMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    areas = []
    
    if len(contours) > 0:
        
        for contour in contours:
            
            areas.append(cv2.contourArea(contour))
        
        maxCon = max(areas)
        
        # Finding the biggest contour found by OpenCV
        
        res = contours[areas.index(maxCon)]
    
        x, y, w, h = cv2.boundingRect(res)
               
        
        try:
            
            # Adding padding around the box of interest. This will fail if it is too close to the edge.
            
            handBox = frame[int(y-h/4):int(y+h*1.25), int(x-w/4):int(x+w*1.25)]
            
            
        except:
            
            # If the hand is at the edge of the screen no padding is applied
            
            handBox = frame[y:y+h, x:x+w]
            
        
        try:
            
            # If the user's hand is too close to the screen, 
            # the box around the hand that was generated by 
            # OpenCV will be taller than the frame, which causes this to fail.
            
            cv2.imshow("Box", handBox)
            
        except:
            
            print("Too close")
        
        
        # Adding the box around the hand
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                
            
        # saveImage() is used for collecting images for training the classifier. 
        # We only ever want to be either saving the images or classifying them directly.
        
        save = false
        
        if save:
            
            saveImage(handBox)
            
        else:
            
            # Two outputs from the classifier function - a verbose version and a long version.
            outputText, combinedOutput = classifyHand(handBox)

            # Printing the results of the classifier in the top left corner.
            # This is looped so that more classes can be added to the model in the future.
            spacing = 25
            for i in range(len(combinedOutput)):
                cv2.putText(frame, combinedOutput[i], (10, 25 + (i * spacing)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)

            # Adding the label on the hand itself.
            cv2.putText(frame, outputText, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
        
        
    cv2.imshow("Output", frame)
        
    
camera.release()
cv2.destroyAllWindows()

Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
Too close
Mitigating error
